In [1]:
##### Impor Pustaka & Konfigurasi Path #####

# ============== LANGKAH 1: INISIALISASI & SETUP =============
import os
import sys
import glob
import cv2
import numpy as np
import json
import time
import torch
from collections import defaultdict
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from deepface import DeepFace
from gfpgan import GFPGANer
import pyiqa

print("Semua pustaka berhasil diimpor.")

# Definisikan kelas Encoder kustom untuk menangani tipe data NumPy
class NumpyJSONEncoder(json.JSONEncoder):
    """ Custom encoder for numpy data types """
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj) # Konversi integer NumPy ke int Python
        elif isinstance(obj, (np.floating, np.float_, np.float16,
                              np.float32, np.float64)):
            return float(obj) # Konversi float NumPy ke float Python
        elif isinstance(obj, (np.bool_)):
            return bool(obj) # Konversi boolean NumPy ke bool Python
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist() # Konversi array NumPy ke list Python

        # Biarkan encoder default menangani tipe data lainnya
        return super(NumpyJSONEncoder, self).default(obj)

# --- Konfigurasi Path ---
BASE_DIR = os.path.abspath('.')
# GALLERY_PATH = os.path.join(BASE_DIR, 'data', 'gallery6.2') # Tidak diperlukan di v6.4
PROBES_PATH = os.path.join(BASE_DIR, 'data', 'probes')
RESULTS_PATH = os.path.join(BASE_DIR, 'results_v6.4_extraction') # Path untuk output gambar
CACHE_PATH = os.path.join(BASE_DIR, 'cache_v6.4_extraction') # Path cache jika diperlukan
PROBE_FEATURES_PATH = os.path.join(BASE_DIR, 'features_v6.4') # Path untuk menyimpan fitur probe

# Pastikan semua direktori ada
for path in [RESULTS_PATH, CACHE_PATH, PROBE_FEATURES_PATH]:
    os.makedirs(path, exist_ok=True)
print("Direktori telah disiapkan.")

# --- Setup Device & Pemuatan Model ---
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Menggunakan device: {DEVICE}')

# Muat GFPGAN
print("Memuat model GFPGAN v1.4...")
# Sesuaikan path ini jika perlu
gfpgan_model_path = r'D:\\UNSRI_DATA\\_SKRIPSI\\PROGRAM\\v1\\model_gfpgan\\gfpgan\\weights\\GFPGANv1.4.pth'
gfpgan_restorer = GFPGANer(model_path=gfpgan_model_path, upscale=2, arch='clean',
                           channel_multiplier=2, bg_upsampler=None, device=DEVICE)
print("Model GFPGAN siap.")

# Muat IQA Assessors
print("Memuat model NR-IQA...")
brisque_assessor = pyiqa.create_metric('brisque', device=DEVICE)
niqe_assessor = pyiqa.create_metric('niqe', device=DEVICE)
print("Model BRISQUE & NIQE siap.")

# Warm-up DeepFace
print("Warm-up model DeepFace (ArcFace)...")
_ = DeepFace.represent(np.zeros((112, 112, 3), dtype=np.uint8), model_name='ArcFace', enforce_detection=False)
print("Model DeepFace siap.")


Semua pustaka berhasil diimpor.
Direktori telah disiapkan.
Menggunakan device: cuda
Memuat model GFPGAN v1.4...
Model GFPGAN siap.
Memuat model NR-IQA...
Model BRISQUE & NIQE siap.
Warm-up model DeepFace (ArcFace)...
Model DeepFace siap.


In [2]:
# ============== LANGKAH 2: DEFINISI FUNGSI UTILITAS =============

def parse_filename(filename: str) -> dict | None:
    """Menganalisis nama file probe untuk mendapatkan metadata."""
    try:
        base_name = os.path.basename(filename)
        parts = os.path.splitext(base_name)[0].split('_')
        # Asumsi format: 'subject_X_height_Y_distance_Z.JPG'
        if len(parts) >= 5:
             subject_id = parts[0]
             height_id = parts[2]
             distance_id = parts[4]
             # Konversi ID jarak ke meter (sesuaikan logika ini jika perlu)
             distance_m = 17 - (int(distance_id) / 2)
             # Konversi ID ketinggian ke meter (sesuaikan logika ini jika perlu)
             height_m = 1.5 if height_id == "0" else int(height_id)
             return {'subject_id': subject_id, 'height_m': height_m, 'distance_m': distance_m}
        return {'subject_id': parts[0]} # Fallback jika format tidak lengkap
    except (IndexError, ValueError):
        # Tangani error jika nama file tidak sesuai format atau konversi gagal
        try:
             # Coba parsing hanya subject_id jika format lengkap gagal
             base_name = os.path.basename(filename)
             parts = os.path.splitext(base_name)[0].split('_')
             return {'subject_id': parts[0]}
        except:
            print(f"Peringatan: Gagal mem-parsing nama file: {filename}")
            return None

# Fungsi ini mungkin tidak digunakan langsung di loop utama v6.4,
# tapi bisa berguna untuk analisis lain atau jika ingin memproses probe non-restorasi
def get_embedding(image_path_or_array, model_name='ArcFace', detector_backend='retinaface') -> list | None:
    """Mengekstrak embedding wajah dari path gambar atau array (dengan deteksi)."""
    try:
        embedding_objs = DeepFace.represent(
            img_path=image_path_or_array, model_name=model_name,
            enforce_detection=True, detector_backend=detector_backend
        )
        # DeepFace.represent bisa mengembalikan list jika ada >1 wajah, ambil yg pertama
        if embedding_objs and isinstance(embedding_objs, list):
            return embedding_objs[0]['embedding']
        return None # Tidak ada wajah terdeteksi atau format output tidak sesuai
    except Exception as e:
        # print(f"Error saat get_embedding: {e}") # Uncomment untuk debug
        return None

def get_embedding_from_cropped(cropped_face_array, model_name='ArcFace') -> list | None:
    """Mengekstrak embedding dari array gambar wajah yang sudah di-crop (tanpa deteksi ulang)."""
    if cropped_face_array is None or cropped_face_array.size == 0:
        return None
    try:
        # Pastikan input adalah BGR numpy array
        if not isinstance(cropped_face_array, np.ndarray):
             return None

        embedding_objs = DeepFace.represent(
            img_path=cropped_face_array, model_name=model_name,
            enforce_detection=False # Penting: tidak perlu deteksi ulang
        )
        # Asumsi output selalu list, ambil embedding pertama
        if embedding_objs and isinstance(embedding_objs, list):
            return embedding_objs[0]['embedding']
        return None # Format output tidak sesuai
    except Exception as e:
        # print(f"Error saat get_embedding_from_cropped: {e}") # Uncomment untuk debug
        return None

def get_nr_iqa_score(image_array, assessor):
    """Menghitung skor kualitas gambar (BRISQUE/NIQE) dari array gambar."""
    if image_array is None or image_array.size == 0: return None
    try:
        # Pastikan input adalah BGR numpy array
        if not isinstance(image_array, np.ndarray): return None

        # Konversi ke RGB dan tensor PyTorch
        img_rgb = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
        # Pastikan tipe data tensor adalah float dan normalisasi ke [0, 1]
        img_tensor = torch.from_numpy(img_rgb).permute(2, 0, 1).unsqueeze(0).float() / 255.0
        score = assessor(img_tensor.to(DEVICE)).item()
        return score
    except Exception as e:
        # print(f"Error saat get_nr_iqa_score: {e}") # Uncomment untuk debug
        return None

def save_comparison_image(original_img, restored_img, save_path):
    """Menyimpan gambar perbandingan (probe vs restorasi) secara berdampingan."""
    if original_img is None or restored_img is None:
        return False
    try:
        target_height = max(original_img.shape[0], restored_img.shape[0])

        def resize_keep_aspect(img):
            if img.shape[0] == target_height:
                return img
            scale = target_height / img.shape[0]
            new_width = int(img.shape[1] * scale)
            return cv2.resize(img, (new_width, target_height), interpolation=cv2.INTER_AREA)

        original_resized = resize_keep_aspect(original_img)
        restored_resized = resize_keep_aspect(restored_img)
        comparison = np.hstack([original_resized, restored_resized])
        cv2.imwrite(save_path, comparison)
        return True
    except Exception as e:
        print(f"Error saat membuat gambar perbandingan: {e}")
        return False

print("Fungsi-fungsi utilitas siap digunakan.")

Fungsi-fungsi utilitas siap digunakan.


In [3]:
# ============== LANGKAH 3: PEMROSESAN PROBE & EKSTRAKSI FITUR =============

# Buat direktori untuk menyimpan gambar output (jika ingin disimpan terpisah)
CROPPED_FACES_PATH = os.path.join(RESULTS_PATH, 'cropped_faces')
RESTORED_FACES_PATH = os.path.join(RESULTS_PATH, 'restored_faces')
RESTORED_IMGS_PATH = os.path.join(RESULTS_PATH, 'restored_imgs')
COMPARISON_PATH = os.path.join(RESULTS_PATH, 'cmp')

for path in [CROPPED_FACES_PATH, RESTORED_FACES_PATH, RESTORED_IMGS_PATH, COMPARISON_PATH]:
    os.makedirs(path, exist_ok=True)

probe_files = glob.glob(os.path.join(PROBES_PATH, '*.JPG')) # Sesuaikan ekstensi jika perlu
probe_features = [] # List untuk menyimpan hasil ekstraksi fitur
print(f"Memulai pemrosesan {len(probe_files)} citra probe...")

for probe_path in tqdm(probe_files, desc="Memproses Probe"):
    metadata = parse_filename(probe_path)
    # Langkahi file jika gagal parsing nama file
    if not metadata or 'subject_id' not in metadata:
        print(f"Peringatan: Melangkahi file karena gagal parsing metadata: {probe_path}")
        continue

    probe_filename = os.path.basename(probe_path)
    ground_truth_label = metadata.get('subject_id', 'unknown') # Ambil subject_id dengan aman

    # Inisialisasi hasil untuk file probe ini
    feature_entry = {
        'file': probe_filename,
        'ground_truth': ground_truth_label,
        'metadata': metadata, # Simpan semua metadata yang berhasil diparsing
        'restoration_succeeded': False,
        'brisque_original': None,
        'niqe_original': None,
        'brisque_restored': None,
        'niqe_restored': None,
        'embedding_original': None,
        'embedding_restored': None
        }

    # --- Baca gambar probe ---
    try:
        # Gunakan imdecode untuk menangani path dengan karakter non-ASCII
        img_probe = cv2.imdecode(np.fromfile(probe_path, np.uint8), cv2.IMREAD_COLOR)
        if img_probe is None:
            print(f"Peringatan: Gagal membaca gambar probe: {probe_path}")
            probe_features.append(feature_entry) # Simpan entri dengan data kosong
            continue
    except Exception as e:
        print(f"Error saat membaca {probe_path}: {e}")
        probe_features.append(feature_entry)
        continue

    # --- Hitung IQA Original ---
    feature_entry['brisque_original'] = get_nr_iqa_score(img_probe, brisque_assessor)
    feature_entry['niqe_original'] = get_nr_iqa_score(img_probe, niqe_assessor)

    # --- Ekstrak Embedding dari Citra Asli ---
    embedding_original = get_embedding(img_probe, model_name='ArcFace', detector_backend='retinaface')
    if embedding_original:
        feature_entry['embedding_original'] = embedding_original
    else:
        print(f"Peringatan: Gagal ekstrak embedding dari citra asli {probe_filename}")

    # --- Lakukan Restorasi dengan GFPGAN ---
    try:
        _, restored_faces, restored_pasted_img = gfpgan_restorer.enhance(
            img_probe,
            has_aligned=True, 
            only_center_face=False # Ambil semua wajah, proses yg pertama
            )

        # Cek apakah restorasi berhasil dan menghasilkan wajah
        if restored_faces and restored_faces[0] is not None:
            restored_face = restored_faces[0] # Ambil wajah pertama yang direstorasi
            feature_entry['restoration_succeeded'] = True

            # --- Hitung IQA Restored ---
            feature_entry['brisque_restored'] = get_nr_iqa_score(restored_face, brisque_assessor)
            feature_entry['niqe_restored'] = get_nr_iqa_score(restored_face, niqe_assessor)

            # --- Ekstrak Embedding dari Wajah Hasil Restorasi ---
            embedding_B = get_embedding_from_cropped(restored_face, model_name='ArcFace')
            if embedding_B:
                feature_entry['embedding_restored'] = embedding_B
            else:
                print(f"Peringatan: Gagal ekstrak embedding dari wajah restorasi {probe_filename}")

            cv2.imwrite(os.path.join(RESTORED_FACES_PATH, probe_filename), restored_face)
            cv2.imwrite(os.path.join(CROPPED_FACES_PATH, probe_filename), restored_face)

            comparison_source = restored_pasted_img if restored_pasted_img is not None else restored_face
            cmp_save_path = os.path.join(COMPARISON_PATH, probe_filename)
            if not save_comparison_image(img_probe, comparison_source, cmp_save_path):
                print(f"Peringatan: Gagal menyimpan gambar perbandingan untuk {probe_filename}")

            if restored_pasted_img is not None:
                cv2.imwrite(os.path.join(RESTORED_IMGS_PATH, probe_filename), restored_pasted_img)
        else:
             print(f"Peringatan: Restorasi GFPGAN tidak menghasilkan wajah untuk {probe_filename}")

    except Exception as enhance_e:
        print(f"Error saat proses enhance/embedding GFPGAN untuk {probe_filename}: {enhance_e}")

    probe_features.append(feature_entry)

# --- Simpan Hasil Ekstraksi Fitur ---
features_file_path = os.path.join(PROBE_FEATURES_PATH, 'probe_features_v6.4.json')
try:
    with open(features_file_path, 'w', encoding='utf-8') as f: # Gunakan encoding utf-8
        # Gunakan NumpyJSONEncoder untuk menyimpan embedding (list of floats)
        json.dump(probe_features, f, indent=4, cls=NumpyJSONEncoder)
    print(f"\nFitur probe berhasil diekstrak dan disimpan ke: {features_file_path}")
    print(f"Jumlah data fitur yang disimpan: {len(probe_features)}")
except Exception as save_e:
    print(f"\nERROR saat menyimpan hasil fitur: {save_e}")

Memulai pemrosesan 1364 citra probe...


Memproses Probe:   0%|          | 0/1364 [00:00<?, ?it/s]

Peringatan: Gagal ekstrak embedding dari citra asli c_gp_5_ef_28.jpg
Peringatan: Gagal ekstrak embedding dari citra asli c_gp_5_ef_29.jpg
Peringatan: Gagal ekstrak embedding dari citra asli e_gp_5_ef_10.jpg
Peringatan: Gagal ekstrak embedding dari citra asli f_gp_5_ef_27.jpg
Peringatan: Gagal ekstrak embedding dari citra asli f_gp_5_ef_28.jpg
Peringatan: Gagal ekstrak embedding dari citra asli f_gp_5_ef_29.jpg
Peringatan: Gagal ekstrak embedding dari citra asli f_gp_5_ef_30.jpg
Peringatan: Gagal ekstrak embedding dari citra asli h_gp_5_ef_02.jpg
Peringatan: Gagal ekstrak embedding dari citra asli i_gp_0_ef_15.jpg
Peringatan: Gagal ekstrak embedding dari citra asli i_gp_3_ef_00.jpg
Peringatan: Gagal ekstrak embedding dari citra asli i_gp_3_ef_02.jpg
Peringatan: Gagal ekstrak embedding dari citra asli k_gp_4_ef_00.jpg
Peringatan: Gagal ekstrak embedding dari citra asli k_gp_5_ef_01.jpg

Fitur probe berhasil diekstrak dan disimpan ke: d:\UNSRI_DATA\_SKRIPSI\PROGRAM\v1\pipeline_skripsi\fea